<a href="https://colab.research.google.com/github/hamidahoderinwale/model_metadata_analyses/blob/main/scraping_hf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Script 1
Takes in a (validated) model url and output its metadata

In [ ]:
# Script 1: takes input model url, validates url, and gives model metadata
!pip install validators
from huggingface_hub import HfApi
import huggingface_hub as hf
import validators
import json
import csv

hf_api = HfApi()

def input_url():
    while True:
        input_model_url = input("Enter model URL: ")
        print(f"You entered: {input_model_url}")

        if validators.url(input_model_url) and "huggingface.co" in input_model_url:
            try:
                # Extract the model ID from the URL
                model_id = input_model_url.split("huggingface.co/")[-1]
                model_info = hf_api.model_info(model_id) # Get model info: https://huggingface.co/docs/huggingface_hub/v0.29.2/en/package_reference/hf_api#huggingface_hub.ModelInfo
                model_card = hf_api.list_models(filter: cardData),
                json_output = json.dumps(model_card.__dict__, indent=4, default=str)
                print(json_output)
                with open('model_info.json', 'w') as json_file:
                    json_file.write(json_output)
            except Exception as e:
                    print(f"Error fetching model info: {str(e)}")
            for key, value in model_info.__dict__.items():
                    print(f"{key}: {value}")
                    return model_info

        else:
            print("Invalid URL. Please enter a valid Hugging Face model URL.") # error code

# Call the function
model_info = input_url()

# To test: https://huggingface.co/deepseek-ai/DeepSeek-R1


Enter model URL: https://huggingface.co/deepseek-ai/DeepSeek-R1
You entered: https://huggingface.co/deepseek-ai/DeepSeek-R1
Error fetching model info: 'function' object has no attribute 'cardData'
id: deepseek-ai/DeepSeek-R1


# Script 2
Takes in a (validated) model and outputs the children models/fine-tunes

In [ ]:
# Script 2
  # 1. Take link as input (format check). This is the "main model"
  # 2. Give the link to the page with the fine-tunes for the inputted model

from huggingface_hub import HfApi
import requests
from bs4 import BeautifulSoup
import re

# Initialize API
api = HfApi()

# Function to validate Hugging Face model URL
def validate_hf_model_url(url):
    pattern = r"^https://huggingface.co/([\w\-]+)/([\w\-]+)$"
    match = re.match(pattern, url)
    if match:
        return match.groups()  # Returns (org/user, model_name)
    return None

# Function to find fine-tuned models
def get_finetuned_models_page(model_org, model_name):
    search_url = f"https://huggingface.co/models?search={model_name}"
    response = requests.get(search_url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        model_links = [
            a["href"] for a in soup.find_all("a", href=True)
            if model_name.lower() in a["href"].lower()
        ]
        return [f"https://huggingface.co{link}" for link in model_links if model_org not in link]

    return []

# Main execution
model_url = input("Enter the Hugging Face model URL: ").strip()

validated = validate_hf_model_url(model_url)
if validated:
    org, model_name = validated
    finetune_links = get_finetuned_models_page(org, model_name)

    if finetune_links:
        print("Fine-tuned models found:")
        for link in finetune_links:
            print(link)
    else:
        print("No fine-tuned models found for this model.")
else:
    print("Invalid Hugging Face model URL format.")



Enter the Hugging Face model URL: https://huggingface.co/perplexity-ai/r1-1776
Fine-tuned models found:
https://huggingface.co/lmstudio-community/r1-1776-GGUF
https://huggingface.co/unsloth/r1-1776-GGUF
https://huggingface.co/unsloth/r1-1776
https://huggingface.co/huihui-ai/r1-1776-GGUF
https://huggingface.co/mlx-community/r1-1776-distill-llama-70b-8bit
https://huggingface.co/mlx-community/r1-1776-distill-llama-70b-4bit
https://huggingface.co/matatonic/r1-1776-distill-llama-70b-6.5bpw-h8-exl2
https://huggingface.co/mmnga/r1-1776-distill-llama-70b-gguf
https://huggingface.co/unsloth/r1-1776-distill-llama-70b
https://huggingface.co/second-state/r1-1776-distill-llama-70b-GGUF
https://huggingface.co/unsloth/r1-1776-distill-llama-70b-GGUF
https://huggingface.co/unsloth/r1-1776-distill-llama-70b-unsloth-bnb-4bit
https://huggingface.co/lmstudio-community/r1-1776-distill-llama-70b-GGUF
https://huggingface.co/mradermacher/r1-1776-distill-llama-70b-i1-GGUF
https://huggingface.co/Apel-sin/r1-1776

# Script 3
## Search steps overview
- `dfs_finetunes` we take the `model_url` as input. Alternatively, we can add this var as an argument.
- We go layer-by-layer and find the children of the current model (i.e. the fine-tunes of a model)
- We call the `dfs_funetunes` function recursively and store the models that have been "visited" to avoid duplicates.
- We have a dictionary of information that we store aboutthe "current model" and have the information stored in respective columns
- We have a `results` list that has the information about all the models and their fine-tunes.

In [ ]:
# Script 3
import requests
import pandas as pd
import datetime
import json
import csv
from huggingface_hub import HfApi
from bs4 import BeautifulSoup
import re

# Initialize API
api = HfApi()

# Function to validate Hugging Face model URL
def validate_hf_model_url(url):
    pattern = r"^https://huggingface.co/([\w\-]+)/([\w\-]+)$"
    match = re.match(pattern, url)
    return match.groups() if match else None


# Function to find fine-tuned models
def get_finetuned_models_page(model_org, model_name):
    search_url = f"https://huggingface.co/models?other=base_model:finetune:{model_org}/{model_name}"
    response = requests.get(search_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        model_divs = soup.find_all("div", class_="w-full truncate")
        model_links = []
        for div in model_divs:
            header = div.find("header")
            if header:
                model_link = header.get("title")
                if model_link:
                    model_links.append(f"https://huggingface.co/{model_link}")
        return model_links
    return []

# Function to get parent model
def get_parent_model(model_url):
    return model_url.split(f"/{model_name}/")[0]

# Truncate metadata
def filter_metadata(json_metadata):
            keys_to_keep = ["modelId", "sha", "tags", "downloads", "pipeline_tag"]
            return {k: json_metadata.get(k) for k in keys_to_keep if k in json_metadata}
            filtered_metadata = filter_metadata(api.model_info(model_id).__dict__)

# Recursive DFS (depth-first search) for finding fine-tunes
def dfs_finetunes(model_url, visited, depth=0, results=None):
       if results is None:
           results = []

       if model_url in visited:
           return results
       visited.add(model_url)

       validated = validate_hf_model_url(model_url)
       if not validated:
           print(f"Invalid URL skipped: {model_url}")
           model_url = "N/A"
           return results

       model_org, model_name = validated
       model_id = f"{model_org}/{model_name}"


       print(f"\n{'  ' * depth}Fetching metadata for: {model_id}")
       try:
           model_metadata = api.model_info(model_id).__dict__
           json_metadata = json.dumps(model_metadata, default=str)

       except Exception as e:
           print(f"Error fetching metadata: {e}")
           return results

       finetune_links = get_finetuned_models_page(model_org, model_name)
       # Removing Duplicate Children
       finetune_links = list(set(finetune_links))
       print(f"{'  ' * depth}Found {len(finetune_links)} fine-tunes at depth {depth}.")

       results.append({
           "model_id": model_id,
           "metadata": json_metadata,
           "depth": depth,
           "children": finetune_links,
           "children_count": len(finetune_links)
       })
       for link in finetune_links:
             dfs_finetunes(link, visited, depth + 1, results)

       return results


# Timestamp for the run
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Function to save results as JSON
def save_json(results, model_name):
    filename = f"{timestamp}_{model_name}_finetunes.json"
    data = {
        "models": results
    }
    with open(filename, "w") as f:
        json.dump(data, f, indent=4, default=str)
    print(f"Results saved to {filename}")

# Function to save results as CSV
''' def save_csv(results, model_name):
    filename = f"{model_name}_{timestamp}_finetunes.csv"
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["model_id", "depth", "children_count", "children", "metadata"])
        writer.writeheader()
        for entry in results:
            # Ensure metadata is a JSON string
            if isinstance(entry["metadata"], dict):
                entry["metadata"] = json.dumps(entry["metadata"], indent=2, default=str)
            # Join children list as a string
            entry["children"] = ", ".join(entry["children"])
            writer.writerow(entry)
    print(f"Results saved to {filename}") '''

# Function to save results as CSV (pandas)
def save_csv(results, model_name):
    filename = f"{model_name}_{timestamp}_finetunes.csv"
    df = pd.DataFrame(results)
    df.to_csv(filename, index=True)
    print(f"Results saved to {filename}")

# Main execution
if __name__ == "__main__":
    model_url = input("Enter the Hugging Face model URL: ").strip()
    visited = set()
    results = dfs_finetunes(model_url, visited)

    if results:
        model_name = results[0]["model_id"].split("/")[-1]  # Extract model name for file naming
        save_json(results, model_name)
        save_csv(results, model_name)
    else:
        print("No fine-tuned models found.")

# Links for testing: https://huggingface.co/NousResearch/DeepHermes-3-Mistral-24B-Preview (4 fine-tunes at depth 0, 0 fine-tunes at depth 1 (all models)), https://huggingface.co/perplexity-ai/r1-1776 (10 fine-tunes at depth 0, 0 fine-tines at depth 1 (all models))


Enter the Hugging Face model URL:  https://huggingface.co/NousResearch/DeepHermes-3-Mistral-24B-Preview

Fetching metadata for: NousResearch/DeepHermes-3-Mistral-24B-Preview


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Found 3 fine-tunes at depth 0.

  Fetching metadata for: Jarrodbarnes/DeepHermes-3-Mistral-24B-Preview-mlx-fp16
  Found 0 fine-tunes at depth 1.

  Fetching metadata for: mlx-community/DeepHermes-3-Mistral-24B-Preview-bf16
  Found 0 fine-tunes at depth 1.

  Fetching metadata for: AlSamCur123/DeepHermes-3-Mistral-24BContinuedFine
  Found 1 fine-tunes at depth 1.

    Fetching metadata for: AlSamCur123/DeepHermes-3-Mistral-24ContinuedFine
    Found 0 fine-tunes at depth 2.
Results saved to 20250404_230058_DeepHermes-3-Mistral-24B-Preview_finetunes.json
Results saved to DeepHermes-3-Mistral-24B-Preview_20250404_230058_finetunes.csv
